In [1]:
# Import the dependencies
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from nltk.corpus import stopwords
# Import CountVectorizer, TfidfVectorizer from sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# Initialize the stopwords
stop_words = set(stopwords.words('english'))
# Import regex
import re
# Create a regex pattern to remove punctuation. 
pattern = r'[^a-zA-Z\s ]'
# Import the pipeline class from the transformers module. 
from transformers import pipeline
# Import gradio 
import gradio as gr
# Import potential requirements to create a Retreival Augmented Generation (RAG) capability
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline

PydanticSchemaGenerationError: Unable to generate pydantic-core schema for <class 'pandas.core.frame.DataFrame'>. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.9/u/schema-for-unknown-type

In [2]:
# Import the SentenceTransformer class and the utility function from the sentence_transformers library.
from sentence_transformers import SentenceTransformer, util
# Use the all-MiniLM-L6-v2 model.
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\mnich\anaconda3\envs\dev\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
# Read the CSV file into a DataFrame.
king_james_bible_df = pd.read_csv('Resources/t_kjv.csv')

In [4]:
# Read the CSV file into a text format.
filepath = "Resources/t_kjv.csv"
with open(filepath) as f:
    bible_text = f.read().replace('\n',' ')

In [5]:
# Print the first 5 lines of the data frame
king_james_bible_df.head()

,id,b,c,v,t
0,1001001,1,1,1,In the beginning God created the heaven and th...
1,1001002,1,1,2,"And the earth was without form, and void; and ..."
2,1001003,1,1,3,"And God said, Let there be light: and there wa..."
3,1001004,1,1,4,"And God saw the light, that it was good: and G..."
4,1001005,1,1,5,"And God called the light Day, and the darkness..."


In [6]:
# Print the first 1000 items in the text
bible_text[0:1000]

'id,b,c,v,t 1001001,1,1,1,In the beginning God created the heaven and the earth. 1001002,1,1,2,"And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters." 1001003,1,1,3,"And God said, Let there be light: and there was light." 1001004,1,1,4,"And God saw the light, that it was good: and God divided the light from the darkness." 1001005,1,1,5,"And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day." 1001006,1,1,6,"And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters." 1001007,1,1,7,"And God made the firmament, and divided the waters which were under the firmament from the waters which were above the firmament: and it was so." 1001008,1,1,8,And God called the firmament Heaven. And the evening and the morning were the second day. 1001009,1,1,9,"And God said, Let the waters under the heaven 

## Potentially relevant Code used in Class

In [7]:
# Define a list of sentences to tokenize.
sentences = ["I love my dog.", "I love my family.", "My dog is a lab."]

## BERT Tokenizer

In [8]:
# Import the BertTokenizer from the transformers package.
from transformers import BertTokenizer

In [9]:
# Instantiate the BertTokenizer on the pre-trained data.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\mnich\anaconda3\envs\dev\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
# Define an input text.
text = "I am learning about subword tokenization."

# Tokenize the text into subwords.
subwords = tokenizer.tokenize(text)
subwords

['i', 'am', 'learning', 'about', 'sub', '##word', 'token', '##ization', '.']

In [11]:
# Initialize the pipeline to translate using the t5-base model. 
translator = pipeline("translation", model="t5-base")

c:\Users\mnich\anaconda3\envs\dev\lib\site-packages\transformers\pipelines\__init__.py:1086: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [12]:
# Define a English text and translate it to German. 
english_text = "I am celebrating my birthday."
text = f"translate English to German: {english_text}"
results = translator(text)
# Display the translation JSON data. 
print(results)
# Get the translated text.
results[0]['translation_text']

[{'translation_text': 'Ich feiere meinen Geburtstag.'}]


'Ich feiere meinen Geburtstag.'

## AutoTokenizer and Translation

In [13]:
# Import the Autotokenizer class from the transformers module. 
from transformers import AutoTokenizer
# Create an instance of the Autotokenizer class using the t5-base model.
tokenizer = AutoTokenizer.from_pretrained("t5-base", max_length=50)

In [14]:
# Define text we want to translate.
english_text = "Hello, how are you today?"

In [15]:
# Retrieve the input IDs from the translation.
input_ids = tokenizer(f"translate English to French: {english_text}", return_tensors="tf").input_ids
input_ids

<tf.Tensor: shape=(1, 13), dtype=int32, numpy=
array([[13959,  1566,    12,  2379,    10,  8774,     6,   149,    33,
           25,   469,    58,     1]])>

In [16]:
# Import the TFAutoModelForSeq2SeqLM class from the transformers module. 
from transformers import TFAutoModelForSeq2SeqLM

In [17]:
# Generate the numerical outputs from the model. 
translation_model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base", max_length=100)
output_ids = translation_model.generate(input_ids, max_new_tokens=100)
output_ids

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


<tf.Tensor: shape=(1, 14), dtype=int32, numpy=
array([[    0, 21845,     6,  1670,   327,     3,  6738,    18,  3249,
         7082,    31,  3464,    58,     1]])>

In [18]:
# Decode the numerical outputs 
tokenizer.decode(output_ids[0])

"<pad> Bonjour, comment vous êtes-vous aujourd'hui?</s>"

In [19]:
# Retrieve the text from the special characters.
tokenizer.decode(output_ids[0], skip_special_tokens=True)

"Bonjour, comment vous êtes-vous aujourd'hui?"

## Text Generation

In [20]:
# Use the text-generation parameter for the pipeline and EleutherAI/gpt-neo-1.3B model. 
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')

In [21]:
# Give the model a prompt. 
prompt = "I like gardening because"
# Pass the prompt to the generator
results = generator(prompt, max_length=125, pad_token_id=50256)
# Get the text based on the prompt. 
generated_text = results[0]['generated_text']
# Print the generated text.
print(generated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


I like gardening because of the outdoors. My home is a garden, we are surrounded, the view is a must. I do not know what to do when it is not my time to be outdoors enjoying the outdoors and being exposed to nature, such as when I was little. My question is, do I have an obligation to plant my yard with flowers for the beauty of it? I live in an area that is not known for wild grasses. I have never heard anyone complain that wild grass was not present at their home. I live in a state known for the large acreage of grass that grow. Do I


In [22]:
# Use the text-generation parameter for the pipeline and EleutherAI/gpt-neo-125m model. 
small_generator = pipeline('text-generation', model='EleutherAI/gpt-neo-125m')

In [23]:
# Give the model a prompt. 
prompt = "My favorite animal is the cat because "
# Pass the prompt to the generator. Use `max_length=25`.
new_results = small_generator(prompt, max_length=25, pad_token_id=50256)
# Get the text based on the prompt. 
generated_text = new_results[0]['generated_text']
# Print the generated text.
print(generated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


My favorite animal is the cat because ive never seen a cat before. ive never seen a cat before. ive


## Question Answering

In [24]:
# Import the pipeline class from the transformers module. 
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

In [25]:
# Source: https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)
text = """
A transformer is a deep learning model that adopts the mechanism of self-attention, differentially weighting the significance of each part of the input data. It is used primarily in the fields of natural language processing (NLP)[1] and computer vision (CV).[2]

Like recurrent neural networks (RNNs), transformers are designed to process sequential input data, such as natural language, with applications towards tasks such as translation and text summarization. However, unlike RNNs, transformers process the entire input all at once. The attention mechanism provides context for any position in the input sequence. For example, if the input data is a natural language sentence, the transformer does not have to process one word at a time. This allows for more parallelization than RNNs and therefore reduces training times.[1]

Transformers were introduced in 2017 by a team at Google Brain[1] and are increasingly becoming the model of choice for NLP problems,[3] replacing RNN models such as long short-term memory (LSTM). The additional training parallelization allows training on larger datasets. This led to the development of pretrained systems such as BERT (Bidirectional Encoder Representations from Transformers) and GPT (Generative Pre-trained Transformer), which were trained with large language datasets, such as the Wikipedia Corpus and Common Crawl, and can be fine-tuned for specific tasks.[4][5]
"""

In [26]:
# Generate a list of questions.
questions = ["When were transformers first introduced?",
             "What are transformers better than?",
             "What are applications of transformers?"]

In [27]:
# Check the output from one question.
question = "When were transformers first introduced?"
# Pass the first question and text to the question_answerer.
result = question_answerer(question=question, context=text)
# Show the results
result

{'score': 0.9091500639915466, 'start': 864, 'end': 868, 'answer': '2017'}

In [28]:
# Create a function to generate the answers based on an input text.
def question_answer(questions, text):
    # Create a list to hold the data that will be added to the DataFrame.
    data = []
    # Use a for loop to iterate through the questions.
    for question in questions:
        # Pass the question and text to the initialized question_answerer. 
        result = question_answerer(question=question, context=text)
        # Retrieve the question, answer, the score, the starting 
        # and ending of where the answer is located in the text.
        data.append([question, result['answer'], result['score'], result['start'], result['end']])
    # Create a DataFrame from the data with appropriate columns. 
    df = pd.DataFrame(data, columns=["Question", "Answer", "Score", "Starting Position", "Ending Position"])
    # Return the DataFrame
    return df

In [29]:
# Call the question_answer function with the questions and text. From Activity 21-2-5
question_answer(questions, text)

,Question,Answer,Score,Starting Position,Ending Position
0,When were transformers first introduced?,2017,0.909150,864,868
1,What are transformers better than?,RNNs,0.575204,481,485
2,What are applications of transformers?,translation and text summarization,0.855785,429,463


## Text Summarization

In [30]:
# Instantiate the pipeline class for summarization using the facebook/bart-large-cnn model.
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

c:\Users\mnich\anaconda3\envs\dev\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [31]:
# Create a variable to contain the text from (https://en.wikipedia.org/wiki/Deep_learning) to summarize.
article ="""Deep learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised.[2] 

Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]

Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains. Specifically, artificial neural networks tend to be static and symbolic, while the biological brain of most living organisms is dynamic (plastic) and analog.[6][7]

The adjective "deep" in deep learning refers to the use of multiple layers in the network. Early work showed that a linear perceptron cannot be a universal classifier, but that a network with a nonpolynomial activation function with one hidden layer of unbounded width can. Deep learning is a modern variation that is concerned with an unbounded number of layers of bounded size, which permits practical application and optimized implementation, while retaining theoretical universality under mild conditions. In deep learning the layers are also permitted to be heterogeneous and to deviate widely from biologically informed connectionist models, for the sake of efficiency, trainability and understandability."""

In [32]:
# Get the most likely summary of the article using "False" for the `do_sample` parameter.
most_likely_summary = summarizer(article, 
                     min_length=30, 
                     max_length=130, 
                     do_sample=False)

# Display the summary
most_likely_summary

[{'summary_text': 'Deep learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design and medical image analysis.'}]

In [33]:
# Get the summary text from the JSON output
most_likely_summary[0]["summary_text"]

'Deep learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design and medical image analysis.'

In [34]:
# Get a more diverse summary of the article using "True" for the `do_sample` parameter.
diverse_summary = summarizer(article, 
                     min_length=30, 
                     max_length=130, 
                     do_sample=True)[0]["summary_text"]

# Display the summary
diverse_summary

'Deep learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation.'

## Using Gradio

## Using Question and Answer with Gradio

In [35]:
# Initialize the pipeline to generate questions and answers using the distilbert-base-cased-distilled-squad model. 
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

In [36]:
# Create a function called `question_answer` that takes two parameters, the text to search and a question.
# The function should return the question, answer, probability score, and the starting and ending index of the answer.
def question_answer(text, question):
    result = question_answerer(question=question, context=text)
    return question, result['answer'], result['score'], result['start'], result['end']

In [37]:
# Create the app with two Textbox components. 
# The first textbox will take the text to search the second will take the question.
# The output should show the question, answer, probability score, and the starting and ending index of the answer.

app = gr.Interface(
    fn=question_answer,
    inputs = [
        gr.Textbox(label="Paste the text to search."), 
        gr.Textbox(label="Ask a question.")],
    outputs=gr.Textbox(lines=10, label="Answer to question, probability score, and location.", show_copy_button=True))
    
# Launch the app.
app.launch(show_error=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Using Summarizer Function with Gradio

In [38]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

c:\Users\mnich\anaconda3\envs\dev\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [39]:
# Create a summary function passing in the desired parameters. 
def summarize(article, max_output):
    return f'{summarizer(article, max_length=max_output, min_length=30, do_sample=False)[0]["summary_text"]}'

In [40]:
# Create an instance of the Gradio Interface application function with parameters. 
app = gr.Interface(fn=summarize, 
                   title="Text Summarizer using Transformers",
                   inputs=["text", "number"], 
                   outputs=gr.Textbox(lines=20, label="Summarized Text Output", show_copy_button=True))
# Launch the app
app.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


## Creating an RAG

In [41]:
# Load your pre-trained Language Model, like BART or T5
model_name = "facebook/bart-large"  # or "google/t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

c:\Users\mnich\anaconda3\envs\dev\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

c:\Users\mnich\anaconda3\envs\dev\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mnich\.cache\huggingface\hub\models--facebook--bart-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [42]:
# Set up a Retrieval System, FAISS, to create a document index for retrieval
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base"
)

NameError: name 'FAISSDocumentStore' is not defined

# May not be able to use FAISS.  It requires Haystack-Farm, which is not compatible with Pydantic >2.0, but Gradio requires Pydantic >2.7.  

In [44]:
# Add a document to document store
docs = [{"content": "Your document text here", "meta": {"source": "Document 1"}}]  
document_store.write_documents(docs)
document_store.update_embeddings(retriever)  # Update the document embeddings

NameError: name 'document_store' is not defined

In [45]:
# Setting up the retrieval and reading pipeline
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")
pipeline = ExtractiveQAPipeline(reader, retriever)

query = "Your query here"
result = pipeline.run(query=query, top_k_retriever=10, top_k_reader=5)

# See retrieved documents and answers
print(result)


NameError: name 'FARMReader' is not defined

In [46]:
# Integrate with the Model for augmented generation
# 
#  # Retrieve documents and combine them into a context
retrieved_context = " ".join([doc['content'] for doc in result['documents']])

# Generate text using the context
input_text = "Your input question here: " + retrieved_context
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(**inputs, max_length=150)

# Decode the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

KeyError: 'documents'